code to match input data with student traffic reports for finetuning

In [20]:
import pandas as pd

# Reading multiple sheets from Excel file with input data
sheets_dict = pd.read_excel('./RTVSlo/Podatki - PrometnoPorocilo_2022_2023_2024.xlsx', engine="openpyxl", sheet_name=None)
print(sheets_dict.keys()) #prints keys

dict_keys(['2022', '2023', '2024'])


In [3]:
# Accessing a specific sheet by name
sheet2022 = sheets_dict['2022'] # pandas dataframe
sheet2023 = sheets_dict['2023']
sheet2024 = sheets_dict['2024']

print(sheet2022.columns)

Index(['LegacyId', 'Datum', 'Operater', 'A1', 'B1', 'C1', 'A2', 'B2', 'C2',
       'TitlePomembnoSLO', 'ContentPomembnoSLO', 'TitleNesreceSLO',
       'ContentNesreceSLO', 'TitleZastojiSLO', 'ContentZastojiSLO',
       'TitleVremeSLO', 'ContentVremeSLO', 'TitleOvireSLO', 'ContentOvireSLO',
       'TitleDeloNaCestiSLO', 'ContentDeloNaCestiSLO', 'TitleOpozorilaSLO',
       'ContentOpozorilaSLO', 'TitleMednarodneInformacijeSLO',
       'ContentMednarodneInformacijeSLO', 'TitleSplosnoSLO',
       'ContentSplosnoSLO'],
      dtype='object')


In [18]:
import re

# Function to remove HTML tags
def remove_html_tags(text):
    if isinstance(text, str):
        return re.sub(r'<[^>]*>', '', text)
    return text

# Process the data
def process_input_data(data):
    processed_data = []
    for index, row in data.iterrows():
        item = {}
        for column in data.columns:
            value = row[column]
            if pd.notna(value):  # Check if the cell is not empty
                item[column] = remove_html_tags(value)
        processed_data.append(item)

    processed_df = pd.DataFrame(processed_data)
    processed_df = processed_df.reindex(columns=data.columns, fill_value=None) # keep the order of columns
    processed_df = processed_df.dropna(axis=1, how='all')  # Drop columns that are completely empty
    processed_df = processed_df.dropna(axis=0, how='all')  # Drop rows that are completely empty
    
    return processed_df

# Process each sheet
processed_data_2022 = process_input_data(sheet2022)
print(processed_data_2022.columns)
processed_data_2023 = process_input_data(sheet2023)
print(processed_data_2023.columns)
processed_data_2024 = process_input_data(sheet2024)
print(processed_data_2024.columns)

Index(['LegacyId', 'Datum', 'Operater', 'A1', 'B1', 'A2', 'B2', 'C2',
       'TitlePomembnoSLO', 'ContentPomembnoSLO', 'TitleNesreceSLO',
       'ContentNesreceSLO', 'TitleZastojiSLO', 'ContentZastojiSLO',
       'TitleVremeSLO', 'ContentVremeSLO', 'TitleOvireSLO', 'ContentOvireSLO',
       'TitleDeloNaCestiSLO', 'ContentDeloNaCestiSLO', 'TitleOpozorilaSLO',
       'ContentOpozorilaSLO', 'TitleMednarodneInformacijeSLO',
       'ContentMednarodneInformacijeSLO', 'TitleSplosnoSLO',
       'ContentSplosnoSLO'],
      dtype='object')
Index(['LegacyId', 'Datum', 'Operater', 'A1', 'B1', 'B2', 'C2',
       'TitlePomembnoSLO', 'ContentPomembnoSLO', 'TitleNesreceSLO',
       'ContentNesreceSLO', 'TitleZastojiSLO', 'ContentZastojiSLO',
       'TitleVremeSLO', 'ContentVremeSLO', 'TitleOvireSLO', 'ContentOvireSLO',
       'TitleDeloNaCestiSLO', 'ContentDeloNaCestiSLO', 'TitleOpozorilaSLO',
       'ContentOpozorilaSLO', 'TitleMednarodneInformacijeSLO',
       'ContentMednarodneInformacijeSLO', 'Tit

In [11]:
# Save the processed data to CSV files
processed_data_2022.to_csv('./Processed/processed_input_2022.csv', index=False, encoding='utf-8-sig')
processed_data_2023.to_csv('./Processed/processed_input_2023.csv', index=False, encoding='utf-8-sig')
processed_data_2024.to_csv('./Processed/processed_input_2024.csv', index=False, encoding='utf-8-sig')

In [24]:
# read from csv output data
rtf_data = pd.read_csv('./Processed/rtf_content_sorted.csv', encoding='utf-8-sig')
print(rtf_data.columns) # Index(['FilePath', 'Datetime', 'Content'], dtype='object')
# convert column Datetime to datetime
rtf_data['Datetime'] = pd.to_datetime(rtf_data['Datetime'], format='%Y-%m-%d %H:%M:%S')
# separate by year
rtf_data_2022 = rtf_data[rtf_data['Datetime'].dt.year == 2022]
rtf_data_2023 = rtf_data[rtf_data['Datetime'].dt.year == 2023]
rtf_data_2024 = rtf_data[rtf_data['Datetime'].dt.year == 2024]
# print the number of rows and first 5 rows
print(rtf_data_2022.shape[0])
print(rtf_data_2022.head())
print(rtf_data_2023.shape[0])
print(rtf_data_2023.head())
print(rtf_data_2024.shape[0])
print(rtf_data_2024.head())

Index(['FilePath', 'Datetime', 'Content'], dtype='object')
8259
                                            FilePath            Datetime  \
0  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 06:00:00   
1  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 06:30:00   
2  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 07:00:00   
3  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 08:00:00   
4  ./RTVSlo/Podatki - rtvslo.si/Promet 2022/Janua... 2022-01-01 08:30:00   

                                             Content  
0  Prometne informacije       01. 01. 2022  \t   ...  
1  Prometne informacije       01. 01. 2022  \t   ...  
2  Prometne informacije       01. 01. 2022  \t   ...  
3  Prometne informacije       01. 01. 2022  \t   ...  
4  Prometne informacije       01. 01. 2022  \t   ...  
9004
                                               FilePath            Datetime  \
8259  ./RTVSlo/Podatki - rtvslo.si/Promet 2023/Janua... 2023-01-1

create json file for each year with following structure:
```
[
    {
        "input": [
            {
                "LegacyId": "...",
                "Datum": "...",
                "Operater": "...",
                "A1": "...",
                "B1": "...",
                "A2": "...",
                "B2": "...",
                "C2": "...",
                "TitlePomembnoSLO": "...",
                "ContentPomembnoSLO": "...",
                "TitleNesreceSLO": "...",
                "ContentNesreceSLO": "...",
                "TitleZastojiSLO": "...",
                "ContentZastojiSLO": "...",
                "TitleVremeSLO": "...",
                "ContentVremeSLO": "...",
                "TitleOvireSLO": "...",
                "ContentOvireSLO": "...",
                "TitleDeloNaCestiSLO": "...",
                "ContentDeloNaCestiSLO": "...",
                "TitleOpozorilaSLO": "...",
                "ContentOpozorilaSLO": "...",
                "TitleMednarodneInformacijeSLO": "...",
                "ContentMednarodneInformacijeSLO": "...",
                "TitleSplosnoSLO": "...",
                "ContentSplosnoSLO": "..."
            }, ...
        ]
        "output": {
            "FilePath": ".\/RTVSlo\/Podatki - rtvslo.si\/Promet ...",
            "Datetime": "...",
            "Content": "...",
            "TitleCategory": "..."
        }
    }
]
```

In [41]:
# create json file for each year with "input" from processed data and "output" from rtf data
# each rtf data row will be matched with the processed data rows from one hour to the time of rtf data
def create_json_file(year, rtf_data, processed_data, time_section_in_hours):
    json_data = []
    processed_data['Datum'] = pd.to_datetime(processed_data['Datum'], format='%Y-%m-%d %H:%M:%S')
    rtf_data['Datetime'] = pd.to_datetime(rtf_data['Datetime'], format='%Y-%m-%d %H:%M:%S')
    for index_rtf, row_rtf in rtf_data.iterrows():
        item = {}
        row_rtf_copy = row_rtf.copy()
        # change Datetime to string to put in json
        row_rtf_copy['Datetime'] = row_rtf_copy['Datetime'].strftime('%Y-%m-%d %H:%M:%S')
        item['output'] = row_rtf_copy.to_dict()
        item['input'] = []
        # get the datetime of the rtf data
        datetime = row_rtf['Datetime']
        # get the processed data rows that are within one hour of the rtf data
        start_time = datetime - pd.Timedelta(hours=time_section_in_hours)
        end_time = datetime
        # change Datum to datetime
        
        filtered_data = processed_data[(processed_data['Datum'] >= start_time) & (processed_data['Datum'] <= end_time)]
        for index, row in filtered_data.iterrows():
            # change Datetime and Datum to string
            row_copy = row.copy()
            row_copy['Datum'] = pd.to_datetime(row_copy['Datum'], format='%Y-%m-%d %H:%M:%S')
            row_copy['Datum'] = row_copy['Datum'].strftime('%Y-%m-%d %H:%M:%S')
            # add only non-empty values to the json
            row_dict = row_copy.to_dict()
            row_dict = {k: v for k, v in row_dict.items() if pd.notna(v)}
            item['input'].append(row_dict)
        json_data.append(item)
    
    return json_data

In [42]:
# create json file and save it
import json
def save_json_file(year, json_data):
    with open(f'./Processed/input_output_all_data_{year}.json', 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)

# create json file for each year
json_data_2022 = create_json_file(2022, rtf_data_2022, processed_data_2022, 1)
save_json_file(2022, json_data_2022)
json_data_2023 = create_json_file(2023, rtf_data_2023, processed_data_2023, 1)
save_json_file(2023, json_data_2023)
json_data_2024 = create_json_file(2024, rtf_data_2024, processed_data_2024, 1)
save_json_file(2024, json_data_2024)

/tmp/ipykernel_82829/2149222538.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtf_data['Datetime'] = pd.to_datetime(rtf_data['Datetime'], format='%Y-%m-%d %H:%M:%S')
/tmp/ipykernel_82829/2149222538.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rtf_data['Datetime'] = pd.to_datetime(rtf_data['Datetime'], format='%Y-%m-%d %H:%M:%S')
/tmp/ipykernel_82829/2149222538.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer